# notebook for create init and true test model

In [1]:
from pytomoatt.checkerboard import Checker

checker_model = Checker('./2_models/model_init_N61_61_61.h5')
# study region
R_earth = 6371.0

r_min = R_earth - 50    # 50 km below the surface   (radial coordinate)
r_max =R_earth + 10    # 10 km above the surface   (radial coordinate)
t_min = 30.0          # 30 degrees latitude
t_max = 32.0          # 32 degrees latitude
p_min = 30.0          # 30 degrees longitude
p_max = 32.0          # 32 degrees longitude

# number of points in the radial, latitudinal, and longitudinal directions
n_rtp = [61,61,61]

checker_model.init_axis([R_earth-r_max, R_earth-r_min], [t_min, t_max], [p_min, p_max], n_rtp)

# perturbation amplitude
vel_pert = 0.06
ani_pert = 0.04


checker_model.checkerboard(
    2, 2, 2,
    pert_vel=0.08, pert_ani=0.04,
    lim_x=[30.5, 31.5], lim_y=[30.5, 31.5], lim_z=[0, 40]
)

checker_model.write('2_models/model_ckb_N{:d}_{:d}_{:d}.h5'.format(*n_rtp))


[30.         30.03333333 30.06666667 30.1        30.13333333 30.16666667
 30.2        30.23333333 30.26666667 30.3        30.33333333 30.36666667
 30.4        30.43333333 30.46666667 30.5        30.53333333 30.56666667
 30.6        30.63333333 30.66666667 30.7        30.73333333 30.76666667
 30.8        30.83333333 30.86666667 30.9        30.93333333 30.96666667
 31.         31.03333333 31.06666667 31.1        31.13333333 31.16666667
 31.2        31.23333333 31.26666667 31.3        31.33333333 31.36666667
 31.4        31.43333333 31.46666667 31.5        31.53333333 31.56666667
 31.6        31.63333333 31.66666667 31.7        31.73333333 31.76666667
 31.8        31.83333333 31.86666667 31.9        31.93333333 31.96666667
 32.        ] 30.5 31.5
15 15 0.03333333333333333
[30.         30.03333333 30.06666667 30.1        30.13333333 30.16666667
 30.2        30.23333333 30.26666667 30.3        30.33333333 30.36666667
 30.4        30.43333333 30.46666667 30.5        30.53333333 30.56666667
 

ValueError: could not broadcast input array from shape (81,) into shape (10,)